In [1]:
import os
os.chdir('../../')

In [2]:
!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Fri Apr 18 22:09:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.04             Driver Version: 570.124.04     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4090        Off |   00000000:02:00.0 Off |                  Off |
| 30%   32C    P8             13W /  450W |      87MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### from "sample pair.ipynb"

In [3]:
!ls samples/64x64_diffusion/dpmsolver++_order3_200

image_samples


In [4]:
import numpy as np
import torch
import torch.nn.functional as F

N = 128
M = 16
K = 20
root_dir = 'samples/64x64_diffusion/dpmsolver++_order3_200/image_samples/images'
npz_file = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if '.npz' in f]

noises = []
images = []
for file in npz_file:
    data = np.load(file)
    noise = data['noise']
    image = data['image']
    noises.append(noise)
    images.append(image)

noises = torch.tensor(noises)
images = torch.tensor(images)
print(noises.shape, images.shape)

torch.Size([128, 3, 64, 64]) torch.Size([128, 3, 64, 64])


/tmp/ipykernel_3930111/3617523202.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  noises = torch.tensor(noises)


In [5]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import random

from main import parse_args_and_config, Diffusion
from datasets import inverse_data_transform

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.benchmark = False



2025-04-18 22:09:13.889993: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-18 22:09:13.909726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-18 22:09:13.909751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-18 22:09:13.910525: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-18 22:09:13.914887: I tensorflow/core/platform/cpu_feature_guar

In [6]:
!mkdir /data/guided-diffusion/scale/rbf_ecp_marginal_64

mkdir: cannot create directory ‘/data/guided-diffusion/scale/rbf_ecp_marginal_64’: File exists


In [9]:
for NFE in [6, 8, 10, 12, 15, 20, 25]:
    for order in [3, 4]:
        ###############################################################################
        # 1) Notebook에서 sys.argv를 직접 설정 (argparse 흉내)
        ###############################################################################
        sys.argv = [
            "main.py",
            "--config", "imagenet64.yml",  # 사용하려는 config
            "--sample",
            "--fid",
            "--dpm_solver_type", "data_prediction",
            "--dpm_solver_order", f"{order}",
            "--skip_type", "logSNR",
            "--ni",

            "--sample_type", "rbf_ecp_marginal",
            "--timesteps", f"{NFE}",
            "--scale_dir", "/data/guided-diffusion/scale/rbf_ecp_marginal_64",
        ]

        ###############################################################################
        # 2) 인자/설정 로드
        ###############################################################################
        args, config = parse_args_and_config()

        ###############################################################################
        # 3) Diffusion 객체 생성 -> 모델 로딩
        ###############################################################################
        diffusion = Diffusion(args, config, rank=0)
        diffusion.prepare_model()
        diffusion.model.eval()

        ###############################################################################
        # 4) 배치(25장) 한 번에 샘플링 -> 5x5 그리드(여백 없이) 시각화
        ###############################################################################
        device = diffusion.device

        for k in range(K):
            index = np.random.randint(0, len(noises), size=(M,))
            noise_batch = noises[index].to(device)
            target_batch = images[index].to(device)
            pred, _ = diffusion.sample_image(noise_batch, diffusion.model, target=target_batch, number=k)
            loss = F.mse_loss(pred, target_batch)
            print('NFE :', NFE, 'order :', order, 'loss :', loss)


INFO - main.py - 2025-04-18 22:13:38,143 - Using device: cuda
INFO - main.py - 2025-04-18 22:13:38,143 - Using device: cuda
INFO - main.py - 2025-04-18 22:13:38,143 - Using device: cuda
INFO - main.py - 2025-04-18 22:13:38,143 - Using device: cuda
INFO - main.py - 2025-04-18 22:13:38,143 - Using device: cuda
INFO - main.py - 2025-04-18 22:13:38,143 - Using device: cuda


[prepare_model] Model is ready.
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=3,number=0.npz  saved!
NFE : 6 order : 3 loss : tensor(0.0267, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=3,number=1.npz  saved!
NFE : 6 order : 3 loss : tensor(0.0263, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=3,number=2.npz  saved!
NFE : 6 order : 3 loss : tensor(0.0320, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=3,number=3.npz  saved!
NFE : 6 order : 3 loss : tensor(0.0273, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=3,number=4.npz  saved!
NFE : 6 order : 3 loss : tensor(0.0291, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=3,number=5.npz  saved!
NFE : 6 order : 3 loss : tensor(0.0294, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=3,number=6.npz  saved!
NFE : 6 order : 3 loss : tensor(0.0267, device='cuda:0')
/data/gui

INFO - main.py - 2025-04-18 22:15:08,866 - Using device: cuda
INFO - main.py - 2025-04-18 22:15:08,866 - Using device: cuda
INFO - main.py - 2025-04-18 22:15:08,866 - Using device: cuda
INFO - main.py - 2025-04-18 22:15:08,866 - Using device: cuda
INFO - main.py - 2025-04-18 22:15:08,866 - Using device: cuda
INFO - main.py - 2025-04-18 22:15:08,866 - Using device: cuda
INFO - main.py - 2025-04-18 22:15:08,866 - Using device: cuda


/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=3,number=19.npz  saved!
NFE : 6 order : 3 loss : tensor(0.0276, device='cuda:0')
[prepare_model] Model is ready.
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=4,number=0.npz  saved!
NFE : 6 order : 4 loss : tensor(0.0273, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=4,number=1.npz  saved!
NFE : 6 order : 4 loss : tensor(0.0308, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=4,number=2.npz  saved!
NFE : 6 order : 4 loss : tensor(0.0292, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=4,number=3.npz  saved!
NFE : 6 order : 4 loss : tensor(0.0257, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=4,number=4.npz  saved!
NFE : 6 order : 4 loss : tensor(0.0244, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=6,p=4,number=5.npz  saved!
NFE : 6 order : 4 loss : tensor(0.0310, device='cuda:0')
/data/gu

INFO - main.py - 2025-04-18 22:16:39,464 - Using device: cuda
INFO - main.py - 2025-04-18 22:16:39,464 - Using device: cuda
INFO - main.py - 2025-04-18 22:16:39,464 - Using device: cuda
INFO - main.py - 2025-04-18 22:16:39,464 - Using device: cuda
INFO - main.py - 2025-04-18 22:16:39,464 - Using device: cuda
INFO - main.py - 2025-04-18 22:16:39,464 - Using device: cuda
INFO - main.py - 2025-04-18 22:16:39,464 - Using device: cuda
INFO - main.py - 2025-04-18 22:16:39,464 - Using device: cuda


...

/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=20,p=3,number=19.npz  saved!
NFE : 20 order : 3 loss : tensor(0.0012, device='cuda:0')
[prepare_model] Model is ready.
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=20,p=4,number=0.npz  saved!
NFE : 20 order : 4 loss : tensor(0.0016, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=20,p=4,number=1.npz  saved!
NFE : 20 order : 4 loss : tensor(0.0015, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=20,p=4,number=2.npz  saved!
NFE : 20 order : 4 loss : tensor(0.0022, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=20,p=4,number=3.npz  saved!
NFE : 20 order : 4 loss : tensor(0.0012, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=20,p=4,number=4.npz  saved!
NFE : 20 order : 4 loss : tensor(0.0013, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=20,p=4,number=5.npz  saved!
NFE : 20 order : 4 loss : tensor(0.0033, device='cud

INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - main.py - 2025-04-18 22:52:41,032 - Using device: cuda
INFO - m

/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=20,p=4,number=19.npz  saved!
NFE : 20 order : 4 loss : tensor(0.0011, device='cuda:0')
[prepare_model] Model is ready.
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=3,number=0.npz  saved!
NFE : 25 order : 3 loss : tensor(0.0019, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=3,number=1.npz  saved!
NFE : 25 order : 3 loss : tensor(0.0006, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=3,number=2.npz  saved!
NFE : 25 order : 3 loss : tensor(0.0017, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=3,number=3.npz  saved!
NFE : 25 order : 3 loss : tensor(0.0014, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=3,number=4.npz  saved!
NFE : 25 order : 3 loss : tensor(0.0006, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=3,number=5.npz  saved!
NFE : 25 order : 3 loss : tensor(0.0014, device='cud

INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - main.py - 2025-04-18 22:59:51,312 - Using device: cuda
INFO - m

/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=3,number=19.npz  saved!
NFE : 25 order : 3 loss : tensor(0.0008, device='cuda:0')
[prepare_model] Model is ready.
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=4,number=0.npz  saved!
NFE : 25 order : 4 loss : tensor(0.0015, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=4,number=1.npz  saved!
NFE : 25 order : 4 loss : tensor(0.0013, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=4,number=2.npz  saved!
NFE : 25 order : 4 loss : tensor(0.0012, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=4,number=3.npz  saved!
NFE : 25 order : 4 loss : tensor(0.0008, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=4,number=4.npz  saved!
NFE : 25 order : 4 loss : tensor(0.0009, device='cuda:0')
/data/guided-diffusion/scale/rbf_ecp_marginal_64/NFE=25,p=4,number=5.npz  saved!
NFE : 25 order : 4 loss : tensor(0.0011, device='cud

In [10]:
print('done')

done
